In [1]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import pandas as pd

Using TensorFlow backend.


In [2]:
w, h = 128, 128
def rs_img(img):
    '''W and H is 128 now
    '''
    flatten = [cv2.resize(img[:,:,i], (w, h), interpolation=cv2.INTER_CUBIC) for i in range(img.shape[-1])]
    img = np.array(np.dstack(flatten)) 
    return img

def change_depth(img):
    '''Depth is 16 now
    '''
    img_start = img[:,:,:8]
    
    mid = int(img.shape[-1]/2)
    img_middle = img[:,:,mid-8:mid+8]
    
    img_end = img[:,:,-8:]
    img = np.concatenate((img_start, img_middle, img_end), axis=2)
    return img


def expand_dims(val):
    val_exp = np.expand_dims(val, axis=4)
    return val_exp

def get_number(filename):
    return int(filename[:filename.find('.')])

def sort_paths(paths):
    paths.sort(key = get_number)
    return paths

In [3]:
test_data_path = "E:/ImageClef/clef_dataset/test_images/"

In [4]:
sort_paths(os.listdir(test_data_path))[:5]

['1.npy', '2.npy', '3.npy', '4.npy', '5.npy']

In [5]:
x_test = []
c = 0

for path in sort_paths(os.listdir(test_data_path))[:]:
    img = None
    img = np.load(test_data_path+ "{}".format(path))
    
    # Resize w, h
    img = rs_img(img)
    # Resize depth
    img_rs = change_depth(img)
    
    x_test.append(img_rs)
    c+=1
    print(c, "----", test_data_path+path, img_rs.shape)

x_test = np.array(x_test)
x_test.shape

np.save("x_test_32.npy",x_test)
print("Saved!")

1 ---- E:/ImageClef/clef_dataset/test_images/1.npy (128, 128, 32)
2 ---- E:/ImageClef/clef_dataset/test_images/2.npy (128, 128, 32)
3 ---- E:/ImageClef/clef_dataset/test_images/3.npy (128, 128, 32)
4 ---- E:/ImageClef/clef_dataset/test_images/4.npy (128, 128, 32)
5 ---- E:/ImageClef/clef_dataset/test_images/5.npy (128, 128, 32)
6 ---- E:/ImageClef/clef_dataset/test_images/6.npy (128, 128, 32)
7 ---- E:/ImageClef/clef_dataset/test_images/7.npy (128, 128, 32)
8 ---- E:/ImageClef/clef_dataset/test_images/8.npy (128, 128, 32)
9 ---- E:/ImageClef/clef_dataset/test_images/9.npy (128, 128, 32)
10 ---- E:/ImageClef/clef_dataset/test_images/10.npy (128, 128, 32)
11 ---- E:/ImageClef/clef_dataset/test_images/11.npy (128, 128, 32)
12 ---- E:/ImageClef/clef_dataset/test_images/12.npy (128, 128, 32)
13 ---- E:/ImageClef/clef_dataset/test_images/13.npy (128, 128, 32)
14 ---- E:/ImageClef/clef_dataset/test_images/14.npy (128, 128, 32)
15 ---- E:/ImageClef/clef_dataset/test_images/15.npy (128, 128, 32

In [3]:
x_test = np.load("x_test_32.npy")
x_test.shape

(117, 128, 128, 32)

In [4]:
x_test = expand_dims(x_test)
x_test.shape

(117, 128, 128, 32, 1)

In [5]:
model = None
model = load_model("clef-2.h5")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 128, 128, 32, 1)   0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 127, 127, 31, 32)  288       
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 63, 63, 15, 32)    0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 62, 62, 14, 32)    8224      
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 31, 31, 7, 32)     0         
_______________________________________

In [6]:
y_preds = []
y_preds_arg = []

count = len(x_test)

for i in range(count):
    y = model.predict(np.expand_dims(x_test[i], axis=0))
    y_preds.append(y)
    y_preds_arg.append(np.argmax(y))
    y = None
    
len(y_preds)

117

In [8]:
y_preds

[array([[0.5188091 , 0.48119092]], dtype=float32),
 array([[8.4952284e-05, 9.9991500e-01]], dtype=float32),
 array([[0.2603247, 0.7396753]], dtype=float32),
 array([[0.7886991 , 0.21130094]], dtype=float32),
 array([[0.9206059 , 0.07939404]], dtype=float32),
 array([[0.98788804, 0.01211193]], dtype=float32),
 array([[0.00330089, 0.99669904]], dtype=float32),
 array([[0.9744753 , 0.02552465]], dtype=float32),
 array([[0.80860156, 0.19139846]], dtype=float32),
 array([[0.9988071 , 0.00119288]], dtype=float32),
 array([[0.94970375, 0.05029623]], dtype=float32),
 array([[0.6621292 , 0.33787072]], dtype=float32),
 array([[0.81512153, 0.18487842]], dtype=float32),
 array([[9.9999416e-01, 5.8539422e-06]], dtype=float32),
 array([[9.9953187e-01, 4.6813837e-04]], dtype=float32),
 array([[0.9968347 , 0.00316537]], dtype=float32),
 array([[0.0014552, 0.9985448]], dtype=float32),
 array([[0.7096801 , 0.29031995]], dtype=float32),
 array([[9.9998832e-01, 1.1666248e-05]], dtype=float32),
 array([[9.

In [31]:
y_preds

[array([[0.05571093, 0.9442891 ]], dtype=float32),
 array([[0.19754596, 0.802454  ]], dtype=float32),
 array([[0.648481  , 0.35151902]], dtype=float32),
 array([[9.9988282e-01, 1.1718053e-04]], dtype=float32),
 array([[0.49925074, 0.5007493 ]], dtype=float32),
 array([[0.0414307 , 0.95856935]], dtype=float32),
 array([[0.01684335, 0.9831566 ]], dtype=float32),
 array([[0.8064665, 0.1935335]], dtype=float32),
 array([[0.00112983, 0.9988702 ]], dtype=float32),
 array([[0.01382199, 0.98617804]], dtype=float32),
 array([[0.00442522, 0.99557483]], dtype=float32),
 array([[0.2991311 , 0.70086896]], dtype=float32),
 array([[0.761734, 0.238266]], dtype=float32),
 array([[0.99259865, 0.00740135]], dtype=float32),
 array([[9.999968e-01, 3.173134e-06]], dtype=float32),
 array([[0.79103655, 0.20896345]], dtype=float32),
 array([[0.6096263 , 0.39037374]], dtype=float32),
 array([[0.9918413 , 0.00815864]], dtype=float32),
 array([[0.2115027 , 0.78849727]], dtype=float32),
 array([[0.99193877, 0.0080

In [51]:
#y_preds_arg

In [9]:
dt = pd.read_csv('csvs/TestSet_metaData.csv')
dt.head()

,Filename,md_Disability,md_Relapse,md_SymptomsOfTB,md_Comorbidity,md_Bacillary,md_DrugResistance,md_HigherEducation,md_ExPrisoner,md_Alcoholic,md_Smoking
0,CTR_TST_001.nii.gz,0,1,1,1,0,0,1,0,0,0
1,CTR_TST_002.nii.gz,0,1,0,0,1,4,0,0,0,1
2,CTR_TST_003.nii.gz,0,0,0,0,1,3,0,0,0,1
3,CTR_TST_004.nii.gz,0,1,1,0,0,0,0,0,1,1
4,CTR_TST_005.nii.gz,1,1,0,0,1,3,0,0,0,1


In [10]:
patient_names = dt['Filename'].values
len(patient_names)

117

In [11]:
patient_names

array(['CTR_TST_001.nii.gz', 'CTR_TST_002.nii.gz', 'CTR_TST_003.nii.gz',
       'CTR_TST_004.nii.gz', 'CTR_TST_005.nii.gz', 'CTR_TST_006.nii.gz',
       'CTR_TST_007.nii.gz', 'CTR_TST_008.nii.gz', 'CTR_TST_009.nii.gz',
       'CTR_TST_010.nii.gz', 'CTR_TST_011.nii.gz', 'CTR_TST_012.nii.gz',
       'CTR_TST_013.nii.gz', 'CTR_TST_014.nii.gz', 'CTR_TST_015.nii.gz',
       'CTR_TST_016.nii.gz', 'CTR_TST_017.nii.gz', 'CTR_TST_018.nii.gz',
       'CTR_TST_019.nii.gz', 'CTR_TST_020.nii.gz', 'CTR_TST_021.nii.gz',
       'CTR_TST_022.nii.gz', 'CTR_TST_023.nii.gz', 'CTR_TST_024.nii.gz',
       'CTR_TST_025.nii.gz', 'CTR_TST_026.nii.gz', 'CTR_TST_027.nii.gz',
       'CTR_TST_028.nii.gz', 'CTR_TST_029.nii.gz', 'CTR_TST_030.nii.gz',
       'CTR_TST_031.nii.gz', 'CTR_TST_032.nii.gz', 'CTR_TST_033.nii.gz',
       'CTR_TST_034.nii.gz', 'CTR_TST_035.nii.gz', 'CTR_TST_036.nii.gz',
       'CTR_TST_037.nii.gz', 'CTR_TST_038.nii.gz', 'CTR_TST_039.nii.gz',
       'CTR_TST_040.nii.gz', 'CTR_TST_041.nii.gz', 

In [12]:
names = []

for name in patient_names:
    names.append(name[:-7])

names

['CTR_TST_001',
 'CTR_TST_002',
 'CTR_TST_003',
 'CTR_TST_004',
 'CTR_TST_005',
 'CTR_TST_006',
 'CTR_TST_007',
 'CTR_TST_008',
 'CTR_TST_009',
 'CTR_TST_010',
 'CTR_TST_011',
 'CTR_TST_012',
 'CTR_TST_013',
 'CTR_TST_014',
 'CTR_TST_015',
 'CTR_TST_016',
 'CTR_TST_017',
 'CTR_TST_018',
 'CTR_TST_019',
 'CTR_TST_020',
 'CTR_TST_021',
 'CTR_TST_022',
 'CTR_TST_023',
 'CTR_TST_024',
 'CTR_TST_025',
 'CTR_TST_026',
 'CTR_TST_027',
 'CTR_TST_028',
 'CTR_TST_029',
 'CTR_TST_030',
 'CTR_TST_031',
 'CTR_TST_032',
 'CTR_TST_033',
 'CTR_TST_034',
 'CTR_TST_035',
 'CTR_TST_036',
 'CTR_TST_037',
 'CTR_TST_038',
 'CTR_TST_039',
 'CTR_TST_040',
 'CTR_TST_041',
 'CTR_TST_042',
 'CTR_TST_043',
 'CTR_TST_044',
 'CTR_TST_045',
 'CTR_TST_046',
 'CTR_TST_047',
 'CTR_TST_048',
 'CTR_TST_049',
 'CTR_TST_050',
 'CTR_TST_051',
 'CTR_TST_052',
 'CTR_TST_053',
 'CTR_TST_054',
 'CTR_TST_055',
 'CTR_TST_056',
 'CTR_TST_057',
 'CTR_TST_058',
 'CTR_TST_059',
 'CTR_TST_060',
 'CTR_TST_061',
 'CTR_TST_062',
 'CTR_TS

In [13]:
probab = []

for p in y_preds:
    # probability of HIGH severity as required to make submission
    probab.append(p[0][1])
    
probab

[0.48119092,
 0.999915,
 0.7396753,
 0.21130094,
 0.07939404,
 0.012111927,
 0.99669904,
 0.025524653,
 0.19139846,
 0.0011928844,
 0.05029623,
 0.33787072,
 0.18487842,
 5.853942e-06,
 0.00046813837,
 0.0031653682,
 0.9985448,
 0.29031995,
 1.1666248e-05,
 0.00038143096,
 0.25276646,
 0.99836046,
 0.0071804435,
 0.77039135,
 0.9993043,
 0.8936947,
 0.99703753,
 0.0026066431,
 0.0058977413,
 2.5648296e-05,
 0.35397172,
 0.8958392,
 0.00094338553,
 0.0010845435,
 0.98695105,
 0.9999988,
 0.9994815,
 3.6998346e-05,
 1.1397645e-06,
 0.85585403,
 0.003730467,
 0.03076799,
 0.088085204,
 0.9986123,
 0.9930381,
 0.012927212,
 0.99064165,
 0.2810595,
 0.07009065,
 5.976738e-05,
 0.30723572,
 0.00069352705,
 0.012084971,
 0.99353045,
 1.666003e-05,
 0.00042158997,
 0.0008146294,
 0.6721848,
 0.9999536,
 0.98798966,
 0.9994055,
 0.82840437,
 0.930408,
 0.5214613,
 0.93805224,
 0.00014240482,
 0.014655217,
 0.999141,
 0.3710022,
 0.98294777,
 3.7086077e-06,
 0.9747551,
 0.9979698,
 0.9107998,
 9

In [14]:
len(names), len(probab)

(117, 117)

In [15]:
for n, p in zip(names, probab):
    print(n, p)

CTR_TST_001 0.48119092
CTR_TST_002 0.999915
CTR_TST_003 0.7396753
CTR_TST_004 0.21130094
CTR_TST_005 0.07939404
CTR_TST_006 0.012111927
CTR_TST_007 0.99669904
CTR_TST_008 0.025524653
CTR_TST_009 0.19139846
CTR_TST_010 0.0011928844
CTR_TST_011 0.05029623
CTR_TST_012 0.33787072
CTR_TST_013 0.18487842
CTR_TST_014 5.853942e-06
CTR_TST_015 0.00046813837
CTR_TST_016 0.0031653682
CTR_TST_017 0.9985448
CTR_TST_018 0.29031995
CTR_TST_019 1.1666248e-05
CTR_TST_020 0.00038143096
CTR_TST_021 0.25276646
CTR_TST_022 0.99836046
CTR_TST_023 0.0071804435
CTR_TST_024 0.77039135
CTR_TST_025 0.9993043
CTR_TST_026 0.8936947
CTR_TST_027 0.99703753
CTR_TST_028 0.0026066431
CTR_TST_029 0.0058977413
CTR_TST_030 2.5648296e-05
CTR_TST_031 0.35397172
CTR_TST_032 0.8958392
CTR_TST_033 0.00094338553
CTR_TST_034 0.0010845435
CTR_TST_035 0.98695105
CTR_TST_036 0.9999988
CTR_TST_037 0.9994815
CTR_TST_038 3.6998346e-05
CTR_TST_039 1.1397645e-06
CTR_TST_040 0.85585403
CTR_TST_041 0.003730467
CTR_TST_042 0.03076799
CTR_T

In [16]:
with open('SVRfree-text.txt', 'w') as f:
    for n, p in zip(names, probab):
        print(n,",", p)
        f.write(str(n))
        f.write(",")
        f.write(str(p))
        f.write("\n")
        
f.close()

CTR_TST_001 , 0.48119092
CTR_TST_002 , 0.999915
CTR_TST_003 , 0.7396753
CTR_TST_004 , 0.21130094
CTR_TST_005 , 0.07939404
CTR_TST_006 , 0.012111927
CTR_TST_007 , 0.99669904
CTR_TST_008 , 0.025524653
CTR_TST_009 , 0.19139846
CTR_TST_010 , 0.0011928844
CTR_TST_011 , 0.05029623
CTR_TST_012 , 0.33787072
CTR_TST_013 , 0.18487842
CTR_TST_014 , 5.853942e-06
CTR_TST_015 , 0.00046813837
CTR_TST_016 , 0.0031653682
CTR_TST_017 , 0.9985448
CTR_TST_018 , 0.29031995
CTR_TST_019 , 1.1666248e-05
CTR_TST_020 , 0.00038143096
CTR_TST_021 , 0.25276646
CTR_TST_022 , 0.99836046
CTR_TST_023 , 0.0071804435
CTR_TST_024 , 0.77039135
CTR_TST_025 , 0.9993043
CTR_TST_026 , 0.8936947
CTR_TST_027 , 0.99703753
CTR_TST_028 , 0.0026066431
CTR_TST_029 , 0.0058977413
CTR_TST_030 , 2.5648296e-05
CTR_TST_031 , 0.35397172
CTR_TST_032 , 0.8958392
CTR_TST_033 , 0.00094338553
CTR_TST_034 , 0.0010845435
CTR_TST_035 , 0.98695105
CTR_TST_036 , 0.9999988
CTR_TST_037 , 0.9994815
CTR_TST_038 , 3.6998346e-05
CTR_TST_039 , 1.1397645e-

https://www.crowdai.org/challenges/imageclef-2019-tuberculosis-severity-scoring/submissions